<a href="https://colab.research.google.com/github/project-ccap/project-ccap.github.io/blob/master/notebooks/2021_0531ccap_word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2021年5月版 word2vec デモ

- date: 2021_0510
- filename: 2021_0530ccap_word2vec.pynb
- author: 浅川伸一
- 概要:

In [ ]:
#colab 上で MeCab をインストール際に，次行をコメントアウト
!pip install mecab-python3==0.996.3
import MeCab

!pip install jaconv
import jaconv

import matplotlib.pyplot as plot
!pip install japanize_matplotlib
import japanize_matplotlib 

In [ ]:
#!wget --no-check-certificate --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1B9HGhLZOja4Xku5c_d-kMhCXn1LBZgDb' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1B9HGhLZOja4Xku5c_d-kMhCXn1LBZgDb" -O 2021_05jawiki_hid128_win10_neg10_cbow.bin.gz && rm -rf /tmp/cookies.txt
#!wget --no-check-certificate --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1OWmFOVRC6amCxsomcRwdA6ILAA5s4y4M' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1OWmFOVRC6amCxsomcRwdA6ILAA5s4y4M" -O 2021_05jawiki_hid128_win10_neg10_sgns.bin.gz && rm -rf /tmp/cookies.txt
!wget --no-check-certificate --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1JTkU5SUBU2GkURCYeHkAWYs_Zlbqob0s' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1JTkU5SUBU2GkURCYeHkAWYs_Zlbqob0s" -O 2021_05jawiki_hid200_win20_neg20_cbow.bin.gz && rm -rf /tmp/cookies.txt
#!wget --no-check-certificate --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1VPL2Mr9JgWHik9HjRmcADoxXIdrQ3ds7' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1VPL2Mr9JgWHik9HjRmcADoxXIdrQ3ds7" -O 2021_05jawiki_hid200_win20_neg20_sgns.bin.gz && rm -rf /tmp/cookies.txt

--2021-05-31 11:34:45--  https://docs.google.com/uc?export=download&confirm=8TFk&id=1JTkU5SUBU2GkURCYeHkAWYs_Zlbqob0s
Resolving docs.google.com (docs.google.com)... 172.217.203.102, 172.217.203.138, 172.217.203.100, ...
Connecting to docs.google.com (docs.google.com)|172.217.203.102|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-04-48-docs.googleusercontent.com/docs/securesc/0vtj9953bpq6kle9vkhfft897a0km490/4i3vk0uas7i8641d19cshl0oki8evrl9/1622460825000/10804218431552746902/10045485480160262011Z/1JTkU5SUBU2GkURCYeHkAWYs_Zlbqob0s?e=download [following]
--2021-05-31 11:34:45--  https://doc-04-48-docs.googleusercontent.com/docs/securesc/0vtj9953bpq6kle9vkhfft897a0km490/4i3vk0uas7i8641d19cshl0oki8evrl9/1622460825000/10804218431552746902/10045485480160262011Z/1JTkU5SUBU2GkURCYeHkAWYs_Zlbqob0s?e=download
Resolving doc-04-48-docs.googleusercontent.com (doc-04-48-docs.googleusercontent.com)... 172.217.204.132, 2607:f8b0:400c:c15::84
Conne

In [ ]:
%time
import numpy as np
# 表示精度桁数の設定
np.set_printoptions(suppress=False, formatter={'float': '{:6.3f}'.format})

# word2vec データ処理のため gensim を使う
import os
import sys
from gensim.models import KeyedVectors
from gensim.models import Word2Vec

print('# word2vec データの読み込み')
print('# 訓練済 word2vec，訓練データは wikipedia 全文  読み込みに時間がかかります...', end="")
# ファイルの所在に応じて変更してください
w2v_base = '.'
#w2v_file='2021_05jawiki_hid128_win10_neg10_cbow.bin.gz'
#w2v_file='2021_05jawiki_hid128_win10_neg10_sgns.bin.gz'
w2v_file='2021_05jawiki_hid200_win20_neg20_cbow.bin.gz'
#w2v_file='2021_05jawiki_hid200_win20_neg20_sgns.bin.gz'
asakawa_w2v_file = os.path.join(w2v_base, w2v_file)
asakawa_w2v = KeyedVectors.load_word2vec_format(asakawa_w2v_file, 
                                                encoding='utf-8', 
                                                unicode_errors='replace',
                                                binary=True) 

w2v = asakawa_w2v
print('done')

# word2vec データの読み込み
# 訓練済 word2vec，訓練データは wikipedia 全文  読み込みに時間がかかります...done


In [ ]:
#以下は操作例です。

word = '鬼滅の刃' #入力単語
topn = 10.       #類似度が高い順に何語出力するか
word = input('単語を入力してください')
w2v.most_similar(word)

単語を入力してください花


[('花びら', 0.7862198352813721),
 ('白い花', 0.7861863374710083),
 ('蕾', 0.7830392122268677),
 ('青い花', 0.7601170539855957),
 ('赤い花', 0.7570050954818726),
 ('水仙', 0.7469111680984497),
 ('小さな花', 0.7404030561447144),
 ('牡丹', 0.738473117351532),
 ('梅の花', 0.737271785736084),
 ('咲き', 0.7367553114891052)]

In [ ]:
w2v.most_similar(positive=('王','女'), negative=('男'))

[('聖王', 0.7146469354629517),
 ('ジャラーサンダ', 0.7005405426025391),
 ('王妃', 0.6933296322822571),
 ('臣下', 0.6854950189590454),
 ('君主', 0.6794342994689941),
 ('王子', 0.6732866168022156),
 ('大王', 0.6719251871109009),
 ('皇帝', 0.6606279611587524),
 ('女王', 0.6571927666664124),
 ('大武神王', 0.6570279002189636)]

In [ ]:
w2v.most_similar(positive=('早稲田大学','関西'), negative=('関東'))

[('同志社大学', 0.5012803077697754),
 ('関西学院大学', 0.49142158031463623),
 ('慶應義塾大学', 0.49109768867492676),
 ('関西大学', 0.4891352951526642),
 ('明治大学', 0.48460063338279724),
 ('法政大学', 0.46867406368255615),
 ('立命館大学', 0.46194517612457275),
 ('立教大学', 0.4435693621635437),
 ('青山学院大学', 0.4349278211593628),
 ('日本大学', 0.43411391973495483)]

In [ ]:
w2v.most_similar(positive=('ジャイアンツ','アメリカ合衆国'), negative=('日本'))

[('パイレーツ', 0.4308600425720215),
 ('ブランチ・リッキー', 0.4181841313838959),
 ('カージナルス', 0.4148496985435486),
 ('カブス', 0.4127900004386902),
 ('インディアンス', 0.4110332727432251),
 ('ニューヨーク・ジャイアンツ', 0.40718504786491394),
 ('ミシガン州', 0.40686190128326416),
 ('アスレチックス', 0.4054514169692993),
 ('セントポール・セインツ', 0.4007907509803772),
 ('シンシナティ・レッズ', 0.399544358253479)]

In [ ]:
w2v.most_similar(positive=('鬼滅の刃'))

[('竈門炭治郎', 0.7513207793235779),
 ('約束のネバーランド', 0.6903928518295288),
 ('吾峠呼世晴', 0.689347505569458),
 ('呪術廻戦', 0.6554467082023621),
 ('転生したらスライムだった件', 0.6463882923126221),
 ('ゆらぎ荘の幽奈さん', 0.6366949081420898),
 ('紅蓮の弓矢', 0.6332972645759583),
 ('進撃の巨人', 0.6329469680786133),
 ('僕のヒーローアカデミア', 0.6326904296875),
 ('炎炎ノ消防隊', 0.6274813413619995)]

In [ ]:
w2v.vector_size

200